In [2]:
import pandas as pd
import os

# WNBA Playoff Prediction

## Data Understanding

Importing datasets

In [3]:
dataframes = dict()
for dataset_name in os.listdir('data'):
    if dataset_name.endswith('.csv'):
        dataframes[dataset_name.split(".")[0]] = pd.read_csv('data/' + dataset_name) 

## Data Preparation

### Select Data

Eliminating columns with exclusively unique values

In [4]:
for df_name, df in dataframes.items():
    df_clean = df.copy()
    for col in df.columns:
        if len(pd.unique(df[col])) == 1:
            df_clean.drop(col, axis=1, inplace=True)
    dataframes[df_name] = df_clean


### Integrate Data

Merging all dataframes into a single one

In [5]:
data = pd.merge(dataframes['teams_post'], dataframes['teams'], on=['tmID', 'year'], how='right')
data = pd.merge(data, dataframes['coaches'], on=['tmID', 'year'], suffixes=('_team', '_coach'))
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDWinner', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDLoser', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['players_teams'], on=['tmID', 'year'], suffixes=('_team', '_player'))
data = pd.merge(data, dataframes['players'], left_on='playerID', right_on='bioID')
data = pd.merge(data, dataframes['awards_players'], on=['playerID'], how='left')

In [7]:
data.shape

(2943, 122)

In [14]:
label = "playoff"

0       N
1       Y
2       Y
3       N
4       Y
       ..
2938    N
2939    Y
2940    N
2941    N
2942    Y
Name: playoff, Length: 2943, dtype: object